In [1]:
from __future__ import print_function
from IPython.core.debugger import set_trace
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import config as cf
import numpy as np
import torchvision
import torchvision.transforms as transforms
#import ipdb
import os
import sys
import time
import argparse
import datetime
import scipy.ndimage as ndimage
from networks import *
import random

In [8]:
start_epoch, num_epochs, batch_size, optim_type = cf.start_epoch, cf.num_epochs, cf.batch_size, cf.optim_type
cuda2 = torch.device('cuda:2')

### Data Preparation

In [3]:
gaussian_transforms = [
   transforms.Lambda(lambda x: ndimage.gaussian_filter(x, sigma=0)),
    transforms.Lambda(lambda x: ndimage.gaussian_filter(x, sigma=1)),
    transforms.Lambda(lambda x: ndimage.gaussian_filter(x, sigma=2)),
    transforms.Lambda(lambda x: ndimage.gaussian_filter(x, sigma=5)),
   transforms.Lambda(lambda x: ndimage.gaussian_filter(x, sigma=10))
    ]
transform_train_noise = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(cf.mean['cifar100'], cf.std['cifar100']),
    transforms.RandomChoice(gaussian_transforms),
])
    
transform_train_clean = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(cf.mean['cifar100'], cf.std['cifar100']),
]) # meanstd transformation
    
transform_test_noise = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(cf.mean['cifar100'], cf.std['cifar100']),
    transforms.RandomChoice(gaussian_transforms),
])
transform_test_noise_1 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(cf.mean['cifar100'], cf.std['cifar100']),
    transforms.Lambda(lambda x:ndimage.gaussian_filter(x, sigma=1)),

])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(cf.mean['cifar100'], cf.std['cifar100']),
])



trainset_noise = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train_noise)

trainset_clean = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train_clean)

testset_noise = torchvision.datasets.CIFAR100(root='./data', train=False, download=False, transform=transform_test_noise)

testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=False, transform=transform_test)
testset_noise1 = torchvision.datasets.CIFAR100(root='./data', train=False, download=False, transform=transform_test_noise_1)
num_classes = 100
    
    
    

trainloader_noise = torch.utils.data.DataLoader(trainset_noise, batch_size=batch_size, shuffle=True, num_workers=2)
trainloader_clean = torch.utils.data.DataLoader(trainset_clean, batch_size=batch_size, shuffle=True, num_workers=2)
testloader_noise = torch.utils.data.DataLoader(testset_noise, batch_size=100, shuffle=False, num_workers=2)
testloader_clean = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)
testloader_noise_1 = torch.utils.data.DataLoader(testset_noise1, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


### Network Definition

In [4]:
net = ResNet(50, num_classes).cuda()
net = torch.nn.DataParallel(net, device_ids=[2])
file_name = 'resnet-50'
checkpoint_gauss = torch.load("./checkpoint/cifar100/resnet-50readout_matching_properTransform.t7")
robustNet = checkpoint_gauss['net']
robustNet = torch.nn.DataParallel(robustNet, device_ids=[2])
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'networks.resnet.ResNet_2Read' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'networks.resnet.Bottleneck' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [5]:
def get_sim_loss(layer, matrix_n, matrix_r, eps, lamb = 200, eta = 0.02):
    reg_strength = lamb**(1+layer*eta)
    mn = (1-eps)*matrix_n
    mr= (1-eps)*matrix_r
    loss = ((0.5*torch.log((1+mn)/(1-mn)))- (0.5*torch.log((1+mr)/(1-mr))))**2
    if torch.isnan(loss.mean()):
        set_trace()
    return reg_strength*loss.mean()

In [11]:
def train(epoch):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    optimizer = optim.SGD(net.parameters(), lr=cf.learning_rate(0.1, epoch), momentum=0.9, weight_decay=5e-4)
    print('\n=> Training Epoch #%d, LR=%.4f' %(epoch, cf.learning_rate(0.1, epoch)))
    for batch_idx, (inputs_c, targets_c) in enumerate(trainloader_clean):
        inputs_c, targets_c = inputs_c.cuda().to(cuda2), targets_c.cuda().to(cuda2)
        optimizer.zero_grad()
        (outputs, matrices_reg) = net(inputs_c, compute_similarity=True)
        (_, matrices_rob) = robustNet(inputs_c, img_type="clean", compute_similarity=True)

        loss_similarity = 0.
        for i, (r, g) in enumerate(zip(matrices_reg, matrices_rob)):
            sim_loss = get_sim_loss(i, r,g, 1e-4)
            loss_similarity= loss_similarity + sim_loss
        loss = criterion(outputs, targets_c) + loss_similarity  # Loss
        
        loss.backward()
        optimizer.step() # Optimizer update
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets_c.size(0)
        correct += predicted.eq(targets_c.data).cpu().sum()
        sys.stdout.write('\r')
        sys.stdout.write('| Epoch [%3d/%3d] Iter[%3d/%3d]\t\t Loss: %.4f Acc@1: %.3f%%'
                %(epoch, num_epochs, batch_idx+1,
                    (len(trainset_noise)//batch_size)+1, loss.item(), 100.*correct/total))
        sys.stdout.flush()
    
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct1 = 0
    total1 = 0
    correct2 = 0
    total2 = 0
    for batch_idx, (inputs_n, targets_n) in enumerate(testloader_noise_1):
        inputs_n, targets_n = inputs_n.cuda().to(cuda2), targets_n.cuda().to(cuda2)
        
        outputs_n = net(inputs_n, compute_similarity=False)
        loss = criterion(outputs_n, targets_n)
        
        test_loss += loss.item()
        _, predicted1 = torch.max(outputs_n.data, 1)
        total1 += targets_n.size(0)
        correct1 += predicted1.eq(targets_n.data).cpu().sum()
    acc = 100.*correct1/total1


    print("\n| Validation Epoch #%d\t\t\tLoss (Noise): %.4f Acc@1: %.2f%%" %(epoch, loss.item(), acc))
    
    # Save checkpoint when best model
    if acc > best_acc:
        print('| Saving Best model...\t\t\tTop1 = %.2f%%' %(acc))
        state = {
                'net':net.module if use_cuda else net,
                'acc':acc,
                'epoch':epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        save_point = './checkpoint/'+'cifar100'+os.sep
        if not os.path.isdir(save_point):
            os.mkdir(save_point)
        torch.save(state, save_point+file_name+'_similarity_regularized_layerwiseRegStrength_lambda200_eta0.02_properTransform.t7')
        best_acc = acc

In [12]:
elapsed_time = 0
for epoch in range(start_epoch, start_epoch+num_epochs):
    start_time = time.time()

    train(epoch)
    test(epoch)

    epoch_time = time.time() - start_time
    elapsed_time += epoch_time
    print('| Elapsed time : %d:%02d:%02d'  %(cf.get_hms(elapsed_time)))


=> Training Epoch #1, LR=0.1000
| Epoch [  1/200] Iter[384/391]		 Loss: 10279.6953 Acc@1: 1.000%

KeyboardInterrupt: 

NameError: name 'outputs' is not defined